In [2]:
import h2o
import pandas as pd
import numpy as np

In [3]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [4]:
h2o.init(nthreads=-1)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.201-b09, mixed mode, sharing)


C:\Users\Tu Lam\Anaconda3\lib\site-packages\h2o\backend\server.py:385: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\Users\Tu Lam\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\TULAM~1\AppData\Local\Temp\tmp3i8yxqcl
  JVM stdout: C:\Users\TULAM~1\AppData\Local\Temp\tmp3i8yxqcl\h2o_Tu_Lam_started_from_python.out
  JVM stderr: C:\Users\TULAM~1\AppData\Local\Temp\tmp3i8yxqcl\h2o_Tu_Lam_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_Tu_Lam_nosxgj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


# Load Data

In [5]:
gr_metadata = pd.read_csv('official_goodreads_metadata.csv')
gr_metadata

,asin,average_rating,ratings_count,reviews_count,text_reviews_count,total_ratings_count,total_reviews_count,total_text_reviews_count,publication_year,publication_month,...,publisher,num_pages,format,description,cleaned_description,gr_countDes_before,gr_countDes_after,cleaned_genres,gr_countText_before,gr_countText_after
0,000100039X,4.23,186297,163625,5535,220088,196528,8847,2010.0,1.0,...,Rupa & Co,127.0,Paperback,"Kahlil Gibran's masterpiece, The Prophet, is o...",tahsil vibrants masterpiece prophet one belove...,106.0,66.0,"poetry, fiction, non-fiction",42320,17834
1,0001053655,4.08,16,33,6,676,1552,85,1997.0,NaN,...,HarperCollins,268.0,Hardcover,NaN,NaN,NaN,NaN,"history, historical fiction, biography, non-fi...",158,75
2,0001061240,4.62,10,22,2,221,603,36,1959.0,12.0,...,Western Publishing Company,324.0,Hardcover,NaN,NaN,NaN,NaN,"poetry, children",49,18
3,000161102X,3.86,33,74,4,2929,5786,75,NaN,NaN,...,NaN,190.0,NaN,The snobby girls at the fashionable boarding s...,snobby girl fashionable board school ridicule ...,47.0,25.0,"children, fiction, young-adult, history, histo...",130,61
4,0001711296,4.29,604,1319,48,738,1564,65,NaN,NaN,...,Random House,63.0,NaN,NaN,NaN,NaN,NaN,"children, fiction, poetry, fantasy, paranormal",257,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37228,0312953240,3.80,75,178,8,87,201,8,1995.0,7.0,...,St. Martin's Paperbacks,570.0,Paperback,Recounts the search for and trial of a serial ...,recount search trial serial killer target offr...,41.0,20.0,"mystery, thriller, crime, non-fiction",219,94
37229,0312955138,3.38,27,75,3,29,81,4,1995.0,9.0,...,St. Martin's Paperbacks,320.0,Paperback,"Jill Coit was a voluptuous, dark-skinned beaut...",jill coit voluptuous darkskinned beauty sultry...,156.0,82.0,"mystery, thriller, crime, non-fiction",125,52
37230,0312955154,3.36,52,94,5,56,101,5,1995.0,10.0,...,St. Martin's Press,NaN,Paperback,Paleontologist Cameron Malone has discovered a...,paleontologist cameron alone discover yearly m...,48.0,33.0,"mystery, thriller, crime, fiction",362,184
37231,0312956878,3.78,57,189,3,59,193,4,1995.0,10.0,...,St. Martin's Paperbacks,608.0,Paperback,A KILLER WITHOUT REDEMPTION...\nIn broad dayli...,killer without redemption broad daylight backw...,187.0,103.0,"mystery, thriller, crime, non-fiction",152,76


In [6]:
gr_metadata.loc[gr_metadata['asin'] == '0205739415', 'total_text_reviews_count'] = 1
gr_metadata.loc[gr_metadata['asin'] == '0300084323', 'total_text_reviews_count'] = 1

In [7]:
am_metadata = pd.read_csv('official_amazon_metadata.csv')
am_metadata

,asin,average,rating_count,text_reviews_count,genres,rank,verifiedTrue_count,Format,am_countText_before,am_countText_after
0,0001713353,4.83,54,54,"Childrens Books, Literature & Fiction",1461315,36,"Paperback, Hardcover",2362,1037
1,0001061240,4.87,45,45,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326
2,0001711296,4.44,107,107,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574
3,0002007649,3.37,19,19,"Science & Math, Chemistry",9799524,3,"Kindle Edition, Paperback, Hardcover",5668,2810
4,0001716069,4.61,59,59,"Literature & Fiction, Poetry",3841172,44,"Kindle Edition, Paperback, Hardcover",3081,1457
...,...,...,...,...,...,...,...,...,...,...
37228,0312943636,4.17,24,24,"Literature & Fiction, Fiction",2583900,9,"Mass Market Paperback, Kindle Edition",5228,2508
37229,0312953038,4.00,27,27,"Romance, Historical",553268,12,"Mass Market Paperback, Kindle Edition, Pape...",3184,1420
37230,0312955154,3.29,14,14,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683
37231,0312956878,3.33,15,15,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450


# Prepare Data

In [8]:
am_gr_metadata = pd.merge(gr_metadata[['asin', 'average_rating', 'total_ratings_count', 'total_text_reviews_count', 
                                       'publication_year', 'publication_month', 'publication_day', 'format', 
                                       'cleaned_description', 'gr_countDes_before', 'gr_countDes_after', 'cleaned_genres',
                                       'gr_countText_before', 'gr_countText_after']], 
                          am_metadata, how='inner', on='asin')
am_gr_metadata

,asin,average_rating,total_ratings_count,total_text_reviews_count,publication_year,publication_month,publication_day,format,cleaned_description,gr_countDes_before,...,gr_countText_after,average,rating_count,text_reviews_count,genres,rank,verifiedTrue_count,Format,am_countText_before,am_countText_after
0,000100039X,4.23,220088,8847,2010.0,1.0,1.0,Paperback,tahsil vibrants masterpiece prophet one belove...,106.0,...,17834,4.64,1453,1453,"Literature & Fiction, Poetry",1810945,1130,NaN,69909,31772
1,0001053655,4.08,676,85,1997.0,NaN,NaN,Hardcover,NaN,NaN,...,75,4.48,50,50,Humor & Entertainment,9799161,43,"Kindle Edition, Paperback, Hardcover, Audi...",4888,2240
2,0001061240,4.62,221,36,1959.0,12.0,1.0,Hardcover,NaN,NaN,...,18,4.87,45,45,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326
3,000161102X,3.86,2929,75,NaN,NaN,NaN,NaN,snobby girl fashionable board school ridicule ...,47.0,...,61,4.35,17,17,Literature & Fiction,1542999,13,NaN,788,399
4,0001711296,4.29,738,65,NaN,NaN,NaN,NaN,NaN,NaN,...,117,4.44,107,107,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37228,0312953240,3.80,87,8,1995.0,7.0,15.0,Paperback,recount search trial serial killer target offr...,41.0,...,94,3.69,13,13,"Biographies & Memoirs, True Crime",443719,4,"Mass Market Paperback, Hardcover",2599,1216
37229,0312955138,3.38,29,4,1995.0,9.0,15.0,Paperback,jill coit voluptuous darkskinned beauty sultry...,156.0,...,52,3.58,12,12,"Biographies & Memoirs, True Crime",3470182,6,"Kindle Edition, Hardcover",1489,668
37230,0312955154,3.36,56,5,1995.0,10.0,15.0,Paperback,paleontologist cameron alone discover yearly m...,48.0,...,184,3.29,14,14,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683
37231,0312956878,3.78,59,4,1995.0,10.0,15.0,Paperback,killer without redemption broad daylight backw...,187.0,...,76,3.33,15,15,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450


In [9]:
am_gr_metadata = am_gr_metadata.rename(columns={'average_rating':'gr_average', 'total_ratings_count':'gr_ratings_count',
                                                'total_text_reviews_count':'gr_reviews_count', 
                                                'publication_year':'gr_pub_yr', 'publication_month':'gr_pub_mo', 
                                                'publication_day':'gr_pub_day', 'format':'gr_format',
                                                'cleaned_description':'gr_description', 'cleaned_genres':'gr_genres',
                                                'average':'am_average', 'rating_count':'am_ratings_count', 
                                                'text_reviews_count':'am_reviews_count', 'genres':'am_genres', 
                                                'rank':'am_rank', 'verifiedTrue_count':'am_verifiedTrue_count', 
                                                'Format':'am_format'})
am_gr_metadata

,asin,gr_average,gr_ratings_count,gr_reviews_count,gr_pub_yr,gr_pub_mo,gr_pub_day,gr_format,gr_description,gr_countDes_before,...,gr_countText_after,am_average,am_ratings_count,am_reviews_count,am_genres,am_rank,am_verifiedTrue_count,am_format,am_countText_before,am_countText_after
0,000100039X,4.23,220088,8847,2010.0,1.0,1.0,Paperback,tahsil vibrants masterpiece prophet one belove...,106.0,...,17834,4.64,1453,1453,"Literature & Fiction, Poetry",1810945,1130,NaN,69909,31772
1,0001053655,4.08,676,85,1997.0,NaN,NaN,Hardcover,NaN,NaN,...,75,4.48,50,50,Humor & Entertainment,9799161,43,"Kindle Edition, Paperback, Hardcover, Audi...",4888,2240
2,0001061240,4.62,221,36,1959.0,12.0,1.0,Hardcover,NaN,NaN,...,18,4.87,45,45,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326
3,000161102X,3.86,2929,75,NaN,NaN,NaN,NaN,snobby girl fashionable board school ridicule ...,47.0,...,61,4.35,17,17,Literature & Fiction,1542999,13,NaN,788,399
4,0001711296,4.29,738,65,NaN,NaN,NaN,NaN,NaN,NaN,...,117,4.44,107,107,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37228,0312953240,3.80,87,8,1995.0,7.0,15.0,Paperback,recount search trial serial killer target offr...,41.0,...,94,3.69,13,13,"Biographies & Memoirs, True Crime",443719,4,"Mass Market Paperback, Hardcover",2599,1216
37229,0312955138,3.38,29,4,1995.0,9.0,15.0,Paperback,jill coit voluptuous darkskinned beauty sultry...,156.0,...,52,3.58,12,12,"Biographies & Memoirs, True Crime",3470182,6,"Kindle Edition, Hardcover",1489,668
37230,0312955154,3.36,56,5,1995.0,10.0,15.0,Paperback,paleontologist cameron alone discover yearly m...,48.0,...,184,3.29,14,14,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683
37231,0312956878,3.78,59,4,1995.0,10.0,15.0,Paperback,killer without redemption broad daylight backw...,187.0,...,76,3.33,15,15,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450


In [10]:
am_gr_metadata['rating_diff'] = am_gr_metadata['am_average'] - am_gr_metadata['gr_average']
am_gr_metadata['ratings_count'] = am_gr_metadata['am_ratings_count'] + am_gr_metadata['gr_ratings_count']
am_gr_metadata['reviews_count'] = am_gr_metadata['am_reviews_count'] + am_gr_metadata['gr_reviews_count']

In [11]:
am_gr_metadata

,asin,gr_average,gr_ratings_count,gr_reviews_count,gr_pub_yr,gr_pub_mo,gr_pub_day,gr_format,gr_description,gr_countDes_before,...,am_reviews_count,am_genres,am_rank,am_verifiedTrue_count,am_format,am_countText_before,am_countText_after,rating_diff,ratings_count,reviews_count
0,000100039X,4.23,220088,8847,2010.0,1.0,1.0,Paperback,tahsil vibrants masterpiece prophet one belove...,106.0,...,1453,"Literature & Fiction, Poetry",1810945,1130,NaN,69909,31772,0.41,221541,10300
1,0001053655,4.08,676,85,1997.0,NaN,NaN,Hardcover,NaN,NaN,...,50,Humor & Entertainment,9799161,43,"Kindle Edition, Paperback, Hardcover, Audi...",4888,2240,0.40,726,135
2,0001061240,4.62,221,36,1959.0,12.0,1.0,Hardcover,NaN,NaN,...,45,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326,0.25,266,81
3,000161102X,3.86,2929,75,NaN,NaN,NaN,NaN,snobby girl fashionable board school ridicule ...,47.0,...,17,Literature & Fiction,1542999,13,NaN,788,399,0.49,2946,92
4,0001711296,4.29,738,65,NaN,NaN,NaN,NaN,NaN,NaN,...,107,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574,0.15,845,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37228,0312953240,3.80,87,8,1995.0,7.0,15.0,Paperback,recount search trial serial killer target offr...,41.0,...,13,"Biographies & Memoirs, True Crime",443719,4,"Mass Market Paperback, Hardcover",2599,1216,-0.11,100,21
37229,0312955138,3.38,29,4,1995.0,9.0,15.0,Paperback,jill coit voluptuous darkskinned beauty sultry...,156.0,...,12,"Biographies & Memoirs, True Crime",3470182,6,"Kindle Edition, Hardcover",1489,668,0.20,41,16
37230,0312955154,3.36,56,5,1995.0,10.0,15.0,Paperback,paleontologist cameron alone discover yearly m...,48.0,...,14,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683,-0.07,70,19
37231,0312956878,3.78,59,4,1995.0,10.0,15.0,Paperback,killer without redemption broad daylight backw...,187.0,...,15,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450,-0.45,74,19


In [12]:
am_gr_metadata = am_gr_metadata.drop('am_ratings_count', axis=1)
am_gr_metadata = am_gr_metadata.drop('gr_ratings_count', axis=1)
am_gr_metadata = am_gr_metadata.drop('am_reviews_count', axis=1)
am_gr_metadata = am_gr_metadata.drop('gr_reviews_count', axis=1)
am_gr_metadata

,asin,gr_average,gr_pub_yr,gr_pub_mo,gr_pub_day,gr_format,gr_description,gr_countDes_before,gr_countDes_after,gr_genres,...,am_average,am_genres,am_rank,am_verifiedTrue_count,am_format,am_countText_before,am_countText_after,rating_diff,ratings_count,reviews_count
0,000100039X,4.23,2010.0,1.0,1.0,Paperback,tahsil vibrants masterpiece prophet one belove...,106.0,66.0,"poetry, fiction, non-fiction",...,4.64,"Literature & Fiction, Poetry",1810945,1130,NaN,69909,31772,0.41,221541,10300
1,0001053655,4.08,1997.0,NaN,NaN,Hardcover,NaN,NaN,NaN,"history, historical fiction, biography, non-fi...",...,4.48,Humor & Entertainment,9799161,43,"Kindle Edition, Paperback, Hardcover, Audi...",4888,2240,0.40,726,135
2,0001061240,4.62,1959.0,12.0,1.0,Hardcover,NaN,NaN,NaN,"poetry, children",...,4.87,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326,0.25,266,81
3,000161102X,3.86,NaN,NaN,NaN,NaN,snobby girl fashionable board school ridicule ...,47.0,25.0,"children, fiction, young-adult, history, histo...",...,4.35,Literature & Fiction,1542999,13,NaN,788,399,0.49,2946,92
4,0001711296,4.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"children, fiction, poetry, fantasy, paranormal",...,4.44,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574,0.15,845,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37228,0312953240,3.80,1995.0,7.0,15.0,Paperback,recount search trial serial killer target offr...,41.0,20.0,"mystery, thriller, crime, non-fiction",...,3.69,"Biographies & Memoirs, True Crime",443719,4,"Mass Market Paperback, Hardcover",2599,1216,-0.11,100,21
37229,0312955138,3.38,1995.0,9.0,15.0,Paperback,jill coit voluptuous darkskinned beauty sultry...,156.0,82.0,"mystery, thriller, crime, non-fiction",...,3.58,"Biographies & Memoirs, True Crime",3470182,6,"Kindle Edition, Hardcover",1489,668,0.20,41,16
37230,0312955154,3.36,1995.0,10.0,15.0,Paperback,paleontologist cameron alone discover yearly m...,48.0,33.0,"mystery, thriller, crime, fiction",...,3.29,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683,-0.07,70,19
37231,0312956878,3.78,1995.0,10.0,15.0,Paperback,killer without redemption broad daylight backw...,187.0,103.0,"mystery, thriller, crime, non-fiction",...,3.33,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450,-0.45,74,19


# All Numeric Features

### RF w/o hyperparameters

In [12]:
# Get only numeric columns
am_gr_metadata_numeric = am_gr_metadata[['asin', 'gr_pub_yr', 'gr_pub_mo', 'gr_pub_day', 'gr_countDes_before', 
                                         'gr_countDes_after', 'gr_countText_before', 'gr_countText_after', 'am_rank',
                                         'am_verifiedTrue_count', 'am_countText_before', 'am_countText_after', 'rating_diff',
                                         'ratings_count', 'reviews_count']]

In [13]:
# Convert pandas dataframe to h2o dataframe
numeric_h2o = h2o.H2OFrame(am_gr_metadata_numeric.drop('asin', axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
# Column names for x & y
y = 'rating_diff'
x = list(numeric_h2o.columns)

In [15]:
x.remove(y)
print(x)

['gr_pub_yr', 'gr_pub_mo', 'gr_pub_day', 'gr_countDes_before', 'gr_countDes_after', 'gr_countText_before', 'gr_countText_after', 'am_rank', 'am_verifiedTrue_count', 'am_countText_before', 'am_countText_after', 'ratings_count', 'reviews_count']


In [16]:
# Split dataframe into train, validation, test data
splits = numeric_h2o.split_frame(ratios=[0.7, 0.15], seed=1)

train = splits[0]
valid = splits[1]
test = splits[2]

In [19]:
numeric_h2o.describe()

Rows:37233
Cols:14




,gr_pub_yr,gr_pub_mo,gr_pub_day,gr_countDes_before,gr_countDes_after,gr_countText_before,gr_countText_after,am_rank,am_verifiedTrue_count,am_countText_before,am_countText_after,rating_diff,ratings_count,reviews_count
type,int,int,int,int,int,int,int,int,int,int,int,real,int,int
mins,1900.0,1.0,1.0,1.0,1.0,1.0,0.0,23.0,0.0,79.0,49.0,-2.78,12.0,11.0
mean,2005.6687716553038,6.367763550154703,13.958339282145069,160.68908468209625,89.97844159504115,3233.2207718958966,1429.169956758787,1563019.943947574,104.3968791126144,13710.038702226566,6490.043697794993,0.387908307146886,11630.116590121721,767.5698439556298
maxs,2019.0,12.0,31.0,2124.0,1250.0,1001655.0,443150.0,21544438.0,39851.0,2384912.0,1082212.0,1.99,4726759.0,190533.0
sigma,8.987800183220946,3.3391205812246585,10.365573948063679,92.75944136790304,52.63472027342033,15029.642129405873,6492.465045443952,2092904.8558293763,569.4105747342433,42458.92595675614,19705.822771857624,0.33529835451844286,85719.84577986541,3599.1655406682116
zeros,0,0,0,0,0,0,5,0,120,0,0,144,0,0
missing,1445,2012,2212,1027,1748,0,0,0,0,0,0,0,0,0
0,2010.0,1.0,1.0,106.0,66.0,42320.0,17834.0,1810945.0,1130.0,69909.0,31772.0,0.40999999999999925,221541.0,10300.0
1,1997.0,nan,nan,nan,nan,158.0,75.0,9799161.0,43.0,4888.0,2240.0,0.4000000000000003,726.0,135.0
2,1959.0,12.0,1.0,nan,nan,49.0,18.0,321557.0,30.0,3085.0,1326.0,0.25,266.0,81.0


In [22]:
# Initiate basic model w/o hyperparameters
rf = H2ORandomForestEstimator(seed=1)

In [29]:
# Train
rf.train(x=x, y=y, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [30]:
rf

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1589134309178_2


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,6609991.0,20.0,20.0,20.0,9317.0,11728.0,10543.62




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.10726328329344884
RMSE: 0.32751073767656663
MAE: 0.2408845195204216
RMSLE: NaN
Mean Residual Deviance: 0.10726328329344884

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 0.10035668732324755
RMSE: 0.31679123618441146
MAE: 0.23420063192956184
RMSLE: NaN
Mean Residual Deviance: 0.10035668732324755

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2020-05-10 11:34:30,6.947 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-05-10 11:36:31,2 min 7.892 sec,1.0,0.454323,0.334585,0.206409,0.447378,0.330786,0.200147
2,,2020-05-10 11:40:36,6 min 13.600 sec,3.0,0.424756,0.314033,0.180418,0.363622,0.271760,0.132221
3,,2020-05-10 11:42:05,7 min 42.222 sec,5.0,0.403383,0.298466,0.162718,0.344699,0.258251,0.118817
4,,2020-05-10 11:43:49,9 min 26.830 sec,50.0,0.327511,0.240885,0.107263,0.316791,0.234201,0.100357



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,am_countText_before,8668.500000,1.000000,0.104958
1,am_countText_after,8151.118164,0.940315,0.098694
2,am_rank,7342.374512,0.847018,0.088902
3,am_verifiedTrue_count,7069.053223,0.815487,0.085592
4,gr_countDes_after,6997.346191,0.807215,0.084724
5,gr_countDes_before,6589.741699,0.760194,0.079789
6,gr_pub_yr,6542.543457,0.754749,0.079217
7,gr_pub_day,6484.357422,0.748037,0.078513
8,gr_pub_mo,5651.887695,0.652003,0.068433
9,reviews_count,5650.187988,0.651807,0.068413


In [31]:
rf.model_performance(test_data=test)


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.098944484798108
RMSE: 0.3145544226332035
MAE: 0.23403100988694253
RMSLE: NaN
Mean Residual Deviance: 0.098944484798108


### RF w/ all features

In [34]:
am_gr_metadata.columns

Index(['asin', 'gr_average', 'gr_pub_yr', 'gr_pub_mo', 'gr_pub_day',
       'gr_format', 'gr_description', 'gr_countDes_before',
       'gr_countDes_after', 'gr_genres', 'gr_countText_before',
       'gr_countText_after', 'am_average', 'am_genres', 'am_rank',
       'am_verifiedTrue_count', 'am_format', 'am_countText_before',
       'am_countText_after', 'rating_diff', 'ratings_count', 'reviews_count'],
      dtype='object')

In [13]:
am_gr_metadata_combined = am_gr_metadata[['asin', 'gr_pub_yr', 'gr_pub_mo', 'gr_pub_day', 'gr_format', 'gr_countDes_before',
                                          'gr_countDes_after', 'gr_genres', 'gr_countText_before', 'gr_countText_after',
                                          'am_average', 'am_genres', 'am_rank', 'am_verifiedTrue_count', 'am_format',
                                          'am_countText_before', 'am_countText_after', 'rating_diff', 'ratings_count',
                                          'reviews_count']]

In [14]:
am_gr_metadata_combined

,asin,gr_pub_yr,gr_pub_mo,gr_pub_day,gr_format,gr_countDes_before,gr_countDes_after,gr_genres,gr_countText_before,gr_countText_after,am_average,am_genres,am_rank,am_verifiedTrue_count,am_format,am_countText_before,am_countText_after,rating_diff,ratings_count,reviews_count
0,000100039X,2010.0,1.0,1.0,Paperback,106.0,66.0,"poetry, fiction, non-fiction",42320,17834,4.64,"Literature & Fiction, Poetry",1810945,1130,NaN,69909,31772,0.41,221541,10300
1,0001053655,1997.0,NaN,NaN,Hardcover,NaN,NaN,"history, historical fiction, biography, non-fi...",158,75,4.48,Humor & Entertainment,9799161,43,"Kindle Edition, Paperback, Hardcover, Audi...",4888,2240,0.40,726,135
2,0001061240,1959.0,12.0,1.0,Hardcover,NaN,NaN,"poetry, children",49,18,4.87,"Childrens Books, Literature & Fiction",321557,30,Hardcover,3085,1326,0.25,266,81
3,000161102X,NaN,NaN,NaN,NaN,47.0,25.0,"children, fiction, young-adult, history, histo...",130,61,4.35,Literature & Fiction,1542999,13,NaN,788,399,0.49,2946,92
4,0001711296,NaN,NaN,NaN,NaN,NaN,NaN,"children, fiction, poetry, fantasy, paranormal",257,117,4.44,Literature & Fiction,2884610,69,"Library Binding, VHS Tape, Paperback, Hard...",5667,2574,0.15,845,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37228,0312953240,1995.0,7.0,15.0,Paperback,41.0,20.0,"mystery, thriller, crime, non-fiction",219,94,3.69,"Biographies & Memoirs, True Crime",443719,4,"Mass Market Paperback, Hardcover",2599,1216,-0.11,100,21
37229,0312955138,1995.0,9.0,15.0,Paperback,156.0,82.0,"mystery, thriller, crime, non-fiction",125,52,3.58,"Biographies & Memoirs, True Crime",3470182,6,"Kindle Edition, Hardcover",1489,668,0.20,41,16
37230,0312955154,1995.0,10.0,15.0,Paperback,48.0,33.0,"mystery, thriller, crime, fiction",362,184,3.29,"Literature & Fiction, Fiction",3412599,4,"Kindle Edition, Paperback",1456,683,-0.07,70,19
37231,0312956878,1995.0,10.0,15.0,Paperback,187.0,103.0,"mystery, thriller, crime, non-fiction",152,76,3.33,"Biographies & Memoirs, True Crime",2606128,9,"Mass Market Paperback, Hardcover",968,450,-0.45,74,19


In [37]:
# Convert pandas dataframe to h2o dataframe
combined_h2o = h2o.H2OFrame(am_gr_metadata_combined.drop('asin', axis=1))

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Cannot determine file type. for upload_9f56d8cb3f9764a1e6d067be2cc94fc3
  Request: POST /3/ParseSetup
    data: {'check_header': '1', 'source_frames': '["upload_9f56d8cb3f9764a1e6d067be2cc94fc3"]', 'separator': '44'}


### RF w/ selected features

In [ ]:
h2o.cluster().shutdown(prompt=True)